In [1]:
import re

%matplotlib inline
import matplotlib
import seaborn as sns; sns.set()

datapath = "data/neko.txt.cabocha"

## 40. 係り受け解析結果の読み込み（形態素）
形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．

### 準備
 ```
 brew install cabocha
 cabocha -f1 neko.txt > neko.txt.cabocha
 ```
 
### memo
クラスに`__repr__`を定義するとprint等で呼ばれた時の表示方法を指定できる


In [2]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
    
    def __str__(self):
        return self.surface

In [3]:
prg_morph = re.compile(r"(?P<sur>.+?)\t(?P<pos>[^,]+),(?P<pos1>[^,]+),([^,]+,){4}(?P<base>[^,]+).*")

def gen_sentence():
    morph_list = []
    with open(datapath) as f:
        for line in f:
            res_m = prg_morph.match(line)
            if res_m:
                morph_list.append(Morph(res_m.group("sur"), res_m.group("base"), res_m.group("pos"), res_m.group("pos1")))
            elif line == "EOS\n":
                yield morph_list
                morph_list = []

for i, sentence in enumerate(gen_sentence()):
    if i== 3:
        print(list(map(str,sentence)))
        break
            

['名前', 'は', 'まだ', '無い', '。']


## 41. 係り受け解析結果の読み込み（文節・係り受け）
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

### memo
文節、形態素、文末の判定は正規表現でやる

1. 文節

    - 形態素は以降の行に書いてあるので後でappendするための受け皿をつくっておく。
    - 係り元を覚えておくための辞書を作る。日本語は係り先の方が文の後ろにあるから先読みせず1行ずつ処理可能
    - EOSが2行以上連なってるところが1文にカウントされてしまっている
    
2. 形態素

    4章を丸々流用

In [27]:
class Chunk:
    def __init__(self, dst, srcs):
        self.morphs = []
        self.dst = dst
        self.srcs = srcs
    
    def __str__(self):
        "形態素をくっつけたものを表示(句読点除く)"
        return re.sub('[、。「」]', '', ''.join(map(str, self.morphs)))
    
    def append(self, morph):
        "文節に形態素を追加"
        self.morphs.append(morph)

In [5]:
from collections import defaultdict

prg_morph = re.compile(r"(?P<sur>.+?)\t(?P<pos>[^,]+),(?P<pos1>[^,]+),([^,]+,){4}(?P<base>[^,]+).*")
prg_chunk = re.compile(r"\* (?P<num>\d+) (?P<dst>-?\d+)D \d+/\d+ .*")

def gen_sentence():
    chunk = Chunk(-1, []) # 名前確保用
    sentence, chunk_srcs = [], defaultdict(list)
    with open(datapath) as f:
        for line in f:
            res_m = prg_morph.match(line)
            res_c = prg_chunk.match(line)
            if res_c:
                me, dst = int(res_c.group("num")), int(res_c.group("dst"))
                chunk_srcs[dst].append(me)
                chunk = Chunk(dst, chunk_srcs[me])
                sentence.append(chunk)
            elif res_m:
                chunk.append(Morph(res_m.group("sur"), res_m.group("base"), res_m.group("pos"), res_m.group("pos1")))
            else: # End of sentence
                yield sentence
                sentence, chunk_srcs = [], defaultdict(list)

def dependency_parsing(n):
    counter = 0
    for sentence in gen_sentence():
        if sentence:
            counter += 1
            if counter == n:
                print("sentence",counter)
                for chunk in sentence:
                    print(chunk, "\t->", sentence[chunk.dst] if chunk.dst != -1 else None)

dependency_parsing(8)
            

sentence 8
この 	-> 書生というのは
書生というのは 	-> 話である。
時々 	-> 捕えて
我々を 	-> 捕えて
捕えて 	-> 煮て
煮て 	-> 食うという
食うという 	-> 話である。
話である。 	-> None


## 43. 名詞を含む文節が動詞を含む文節に係るものを抽出
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．



In [6]:
with open("out/43.txt", "w") as f:
    for sentence in gen_sentence():
        for chunk in sentence:
            if "名詞" in [m.pos for m in chunk.morphs] and "動詞" in [m.pos for m in sentence[chunk.dst].morphs]:
                line = '\t'.join(map(str, [chunk, sentence[chunk.dst]]))
                f.write(line + "\n") if chunk.dst != -1 else None

## 44. 係り受け木の可視化
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

## 45. 動詞の格パターンの抽出
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

- 動詞を含む文節において，最左の動詞の基本形を述語とする
- 述語に係る助詞を格とする
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる

### memo
1. 動詞入りの文節を探す
1. 最左動詞を持ってくる
1. 係り元の文節を列挙
1. 係り元の文節内の助詞を抽出、ソート

参考: 助詞[(wikipedia)](https://ja.wikipedia.org/wiki/助詞#格助詞)

In [34]:
from itertools import chain

with open("out/45_cp.txt", "w") as f:
    for sentence in gen_sentence():
        for chunk in sentence:
            morph_verbs = [morph.base for morph in chunk.morphs if morph.pos == "動詞"]
            if morph_verbs:
                predicate, *rest = morph_verbs # 最左動詞
                morph_srcs = chain.from_iterable(sentence[i].morphs for i in chunk.srcs) # list(list(morph))のflatten
                particles = [morph.base for morph in morph_srcs if morph.pos == "助詞"] # 助詞の抽出
                if particles:
                    f.write(''.join([predicate, '\t', ' '.join(sorted(particles)), '\n']))


# 動詞→助詞のペアを列挙
with open("out/45_cp_all.txt", "w") as f:
    for sentence in gen_sentence():
        for chunk in sentence:
            morph_verbs = [morph.base for morph in chunk.morphs if morph.pos == "動詞"]
            if morph_verbs:
                predicate, *rest = morph_verbs
                morph_srcs = chain.from_iterable(sentence[i].morphs for i in chunk.srcs)
                particles = [morph.base for morph in morph_srcs if morph.pos1 == "格助詞"]
                for p in particles:
                    f.write(''.join([predicate, '\t', p, '\n']))

このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

- コーパス中で頻出する述語と格パターンの組み合わせ

```sort out/45_cp_all.txt | sort -f | uniq -c | sort -k 1r,1 -k 2f,2 > out/45_cp_pairs.count```

- 「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）

```grep -E "^する" out/45_cp_all.txt | cut -f 2- | sort | uniq -c | sort -rk 1 | awk '{print $2}' | tr '\n' ' ' > out/45_suru.txt```  
```grep -E "^見る" out/45_cp_all.txt | cut -f 2- | sort | uniq -c | sort -rk 1 | awk '{print $2}' | tr '\n' ' ' > out/45_miru.txt```  
```grep -E "^与える" out/45_cp_all.txt | cut -f 2- | sort | uniq -c | sort -rk 1 | awk '{print $2}' | tr '\n' ' ' > out/45_ataeru.txt```  

## 46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

- 項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
- 述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる

### memo
45に加えて...  
タプルで持って助詞で辞書順ソートした後に各々を分けて列挙

In [35]:
from itertools import chain

with open("out/46.txt", "w") as f:
    for sentence in gen_sentence():
        for chunk in sentence:
            morph_verbs = [morph.base for morph in chunk.morphs if morph.pos == "動詞"]
            if morph_verbs:
                predicate, *rest = morph_verbs
                pscs = [(morph.base, str(sentence[i])) for i in chunk.srcs for morph in sentence[i].morphs if morph.pos == "助詞"]
                pscs.sort(key=lambda t: t[0])
                ps, cs =[p for p, c in pscs], [c for p, c in pscs]
                if pscs:
                    f.write(''.join([predicate, '\t', ' '.join(ps), '\t', ' '.join(cs), '\n']))

## 47. 機能動詞構文のマイニング
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

- 「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
- 述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
- 述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
- 述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）

### memo
46に加えて...  
- 文節が「サ変接続→を」を持つかどうか調べる
- 持ってたらそいつの係り先が動詞を持つか探す、あとは46と同様
- 出力フォーマットを少し修正する

In [36]:
from itertools import chain

with open("out/47.txt", "w") as f:
    for sentence in gen_sentence():
        for chunk in sentence:
            #　サ変->を　の抽出
            find_sw = ''.join(["s" if morph.pos1 == "サ変接続" else "w" if str(morph) == "を" else "x" for morph in chunk.morphs])
            match = re.search("sw", find_sw)        
            if match:
                sahen = chunk.morphs[match.start()].surface
                morph_verbs = [morph.base for morph in sentence[chunk.dst].morphs if morph.pos == "動詞"]
                if morph_verbs:
                    predicate, *rest = morph_verbs
                    pscs = [(morph.base, str(sentence[i])) for i in sentence[chunk.dst].srcs for morph in sentence[i].morphs if morph.pos == "助詞"]
                    pscs.sort(key=lambda t: t[0])
                    ps, cs =[p for p, c in pscs], [c for p, c in pscs]
                    if pscs:
                        f.write(''.join([sahen, 'を', predicate, '\t', ' '.join(ps), '\t', ' '.join(cs), '\n']))